# Evaluation<a name = "Top"></a>

# Quick Links

<ol>
    <li><a href = #setup>Setup</a></li>
    <li><a href = #store>Save File</a></li>
</ol>

## Imports

In [ ]:
import os
import shutil

In [ ]:
import talos as ta
from talos.model import lr_normalizer, early_stopper, hidden_layers

import tensorflow as tf
  
available_gpus = tf.config.experimental.list_physical_devices('GPU')
built_with_cuda = tf.test.is_built_with_cuda()

if not (not available_gpus) & built_with_cuda:
    print("The installed version of TensorFlow {} includes GPU support.\n".format(tf.__version__))
    print("Num GPUs Available: ", len(available_gpus), "\n")
else:
    print("The installed version of TensorFlow {} does not include GPU support.\n".format(tf.__version__))
    
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

from tensorflow.compat.v1.keras import callbacks, backend as K
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.utils import multi_gpu_model
from keras.initializers import glorot_uniform
from keras.optimizers import Adam, Nadam, RMSprop, SGD, Adagrad
from keras.layers.advanced_activations import ReLU, LeakyReLU

from datetime import datetime, timedelta
import time
import pandas as pd
import numpy as np
import shutil

from enum import Enum

from numpy import array

import ntpath

import copy
import re

from numpy.random import seed
seed(1)
tf.random.set_seed(1)

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
config.gpu_options.per_process_gpu_memory_fraction = 0.99
sess = tf.compat.v1.Session(config = config)
K.set_session(sess)

## Hilfsfunktionen

### Enum für Trainingsset

In [ ]:
class TrainingSet(Enum):
    SYNTHETIC = 1
    REAL = 2
    MIXED = 3

### Enum für Label-Typ

In [ ]:
class LabelType(Enum):
    ANGULAR = 1
    STEREOGRAPHIC = 2

### Trainingsset-Typ nach String Converter

In [ ]:
def trainingset_to_string(ts):
    if ts == TrainingSet.SYNTHETIC:
        return 'Synth'
    elif ts == TrainingSet.REAL:
        return 'Real'
    elif ts == TrainingSet.MIXED:
        return 'Mixed'
    else:
        print('Unknown TrainingSet')
        return None

### LabelType nach String

In [ ]:
def labeltype_to_string(lt):
    if lt == LabelType.ANGULAR:
        return 'Angular'
    if lt == LabelType.STEREOGRAPHIC:
        return 'Stereographic'
    else:
        print('Unknown LabelType')
        return None

### Benutzerdefinierte Kostenfunktion & Metrik

In [ ]:
def circular_mse(y_true, y_pred):
    max_error = tf.constant(360, dtype = 'float32')
    return K.mean(K.square(K.minimum(K.abs(y_pred - y_true), max_error - K.abs(y_pred - y_true))), axis = -1)

def circular_mae(y_true, y_pred):
    max_error = tf.constant(360, dtype = 'float32')
    return K.mean(K.minimum(K.abs(y_pred - y_true), K.abs(max_error - K.abs(y_pred - y_true))), axis = -1)

def custom_mae(y_true, y_pred):
    return K.mean(K.abs(y_pred - y_true), axis = -1)

### Datenpipeline für Test

In [ ]:
def setup_data(mode, label_type):
    
    if mode == TrainingSet.SYNTHETIC:
        df = pd.read_csv(_CSV_FILE)
        df_shuffled = df.sample(frac=1, random_state = 1)
        df_test = df_shuffled[df_shuffled.shape[0] - 10000 : df_shuffled.shape[0]]
        
    elif mode == TrainingSet.REAL:
        df_real = pd.read_csv(_CSV_FILE_REAL)
        df_shuffled_real = df_real.sample(frac = 1, random_state = 1)
        df_test_real = df_shuffled_real[df_shuffled_real.shape[0] - 61 : df_shuffled_real.shape[0]]
        df_test = df_test_real
        
    elif mode == TrainingSet.MIXED:
        print('Inferenz auf gemischten Testdaten nicht unbedingt sinnvoll')
        df = pd.read_csv(_CSV_FILE)
        df_shuffled = df.sample(frac = 1, random_state = 1)
        df_test = df_shuffled[df_shuffled.shape[0] - 10000 : df_shuffled.shape[0]]
        
        df_real = pd.read_csv(_CSV_FILE_REAL)
        df_shuffled_real = df_real.sample(frac = 1, random_state = 1)
        df_test_real = df_shuffled_real[df_shuffled_real.shape[0] - 61 : df_shuffled_real.shape[0]]
        df_test = df_test.drop(df_test.index[0 : 61])
        df_test = df_test.append(df_test_real)        
        
    else:
        print('Invalider Modus :: setup_data(>mode<)')

    test_data_generator = ImageDataGenerator(
        rescale = 1./255
    )

    test_generator = test_data_generator.flow_from_dataframe(
        dataframe = df_test,
        directory = _IMAGE_DIR,
        x_col = 'Filename',
        y_col = get_Label_Names(label_type),
        class_mode = 'raw',
        target_size = (224, 224),
        shuffle = False,
        batch_size = 1
    )
    
    return test_generator, df_test

### Modell für Inferenz

In [ ]:
def model_predict(parameters, prediction_runs = 1):
    
    K.clear_session()
    
    test_generator, df_test = setup_data(parameters['dataset_to_use'], parameters['label_type'])

    model = Sequential()
    cnn = VGG16(weights = 'imagenet', include_top = False, input_shape = (224, 224, 3))

    for layer in cnn.layers[:15]:
        layer.trainable = False
        #print(layer.name, layer.trainable)
    
    model.add(cnn)
    
    fc = Sequential()
    fc.add(Flatten(input_shape = model.output_shape[1:])) # (7, 7, 512)
    
    if parameters['activation_function'] == 'leakyrelu':
        activation_layer = LeakyReLU(alpha = parameters['leaky_ReLU_alpha'])
    elif parameters['activation_function'] == 'relu':
        activation_layer = ReLU()  

    fc.add(Dense(units = parameters['first_neuron'], kernel_initializer = glorot_uniform(seed = 1)))
    fc.add(activation_layer)
    if parameters['dropout_rate'] > 0.0:
        fc.add(Dropout(rate = parameters['dropout_rate']))

    hidden_neuron_fraction = parameters['first_neuron']
    for i in range(parameters['hidden_layers']):
        hidden_neuron_fraction = hidden_neuron_fraction // 2
        fc.add(Dense(units = hidden_neuron_fraction, kernel_initializer = glorot_uniform(seed = 1)))
        fc.add(activation_layer)
        if parameters['dropout_rate'] > 0.0:
            fc.add(Dropout(rate = parameters['dropout_rate']))

    fc.add(Dense(units = 2, kernel_initializer = glorot_uniform(seed = 1)))
    
    model.add(fc)
    model.load_weights(parameters['model_to_load'])

    print('Using Optimizer: {} with Learning Rate: {}'.format(parameters['optimizer'][0], parameters['learning_rate']))
    model.compile(
        optimizer = parameters['optimizer'][0](lr = parameters['learning_rate']),
        loss = parameters['loss_function'],
        metrics = get_Reduction_Metric(parameters['reduction_metric'])
    )

    test_generator.reset()

    print("Predicting using these values:\nTest Data: {}\nUsing Loss: {} on Dataset: {}".format(parameters['dataset_to_use'], parameters['loss_function'], parameters['dataset_name']))
    print('LabelType: {} Using Metric: {}'.format(parameters['label_type'], parameters['reduction_metric']))
    
    duration = 0    
    
    for n in range(prediction_runs):
        startTime = datetime.now()
        #predictions = model.predict_generator(generator = test_generator, steps = test_generator.n // test_generator.batch_size, verbose = 0)
        predictions = model.predict(x = test_generator, steps = test_generator.n // test_generator.batch_size, verbose = 0)
        duration = (datetime.now() - startTime).total_seconds()
        time.sleep(1)
    
    time_per_prediction = duration / prediction_runs
    time_per_image = time_per_prediction / len(predictions)
    print("Prediction repeated {} times at a total time of {}sec. \nAverage Time per Prediction: {} sec. Average Time per Image: {} sec".format(prediction_runs, duration, time_per_prediction, time_per_image))

    del model
    return predictions, df_test, duration, time_per_image

### Load Params

In [ ]:
def load_params(dataframe, training_set, model_to_load, dataset, label_type = None, loss_function = None, reduction_metric = None, monitor_value = None):
    params = copy.deepcopy(p)
    params['dataset_to_use'] = training_set
    params['model_to_load'] = model_to_load
    params['dataset_name']: dataset
    # -------------------------------------------------------------
    params['optimizer'] = [make_optimizer(dataframe.loc[0]['optimizer'])]
    params['learning_rate'] = dataframe.iloc[0]['lr']
    params['first_neuron'] = dataframe.iloc[0]['first_neuron']
    params['dropout_rate'] = dataframe.iloc[0]['dropout']
    params['activation_function'] = dataframe.iloc[0]['activation']
    params['leaky_ReLU_alpha'] = dataframe.iloc[0]['leaky_alpha']
    params['hidden_layers'] = dataframe.iloc[0]['hidden_layers']
    # -------------------------------------------------------------
    try:
        params['label_type'] = dataframe.iloc[0]['label_type']
    except:
        print('label_type not available in dataframe, using: ', label_type)
        params['label_type'] = label_type
    try:
        params['loss_function'] = dataframe.iloc[0]['loss_function']
    except:
        print('loss_function not available in dataframe, using: ', loss_function)
        params['loss_function'] = loss_function
    try:
        params['reduction_metric'] = dataframe.iloc[0]['reduction_metric']
    except:
        print('reduction_metric not available in dataframe, using: ', reduction_metric)
        params['reduction_metric'] = reduction_metric
    try:
        params['monitor_value'] = dataframe.iloc[0]['monitor_value']
    except:
        print('monitor_value not available in dataframe, using: ', monitor_value)
        params['monitor_value'] = monitor_value
    
    return params

## Hilfsfunktionen

### Konvertierung ($S_x$, $S_y$) $\rightarrow$ ($\phi$, $\theta$)

In [ ]:
def convert_from_stereographic(sx, sy, r = 1):
    
    l = np.sqrt(sx * sx + sy * sy)
    theta = 90 - 2 * np.degrees(np.arctan(l / 2))
    
    if sx < 0:
        phi = 180 - np.degrees(np.arcsin(sy / l))
        
    elif sx >= 0:
        if sy > 0:
            phi = np.degrees(np.arcsin(sy / l))
        elif sy < 0:
            phi = 360 + np.degrees(np.arcsin(sy / l))
        else:
            #phi1 = np.NaN
            phi = 0
    else:
        print('sx and sy undefined. should not have reached here')

    return phi, theta

### Konvertierung ($\phi$, $\theta$) $\rightarrow$ ($S_x$, $S_y$)

In [ ]:
def convert_from_spheric(phi, theta, r = 1):
    m = 2 * r * np.tan(np.radians((90 - theta) / 2))
    sy = m * np.sin(np.radians(phi))
    sx = m * np.cos(np.radians(phi))
    return sx, sy

### Radians $\rightarrow$ Degree

In [ ]:
def to_degree(angle_in_rad):
    return angle_in_rad * 180 / np.pi

### Degree $\rightarrow$ Radians

In [ ]:
def to_radians(angle_in_deg):
    return angle_in_deg * np.pi / 180

### Sphärische $\rightarrow$ Karthesische Koordinaten

In [ ]:
def spheric_cartesian_polar(phi_d, theta_d):
    x = np.sin(np.radians(90.0 - theta_d)) * np.cos(np.radians(phi_d))
    y = np.sin(np.radians(90.0 - theta_d)) * np.sin(np.radians(phi_d))
    z = np.cos(np.radians(90.0 - theta_d))
    return array([x, y, z])

In [ ]:
def spheric_cartesian_elevation(phi_d, theta_d):
    x = np.cos(np.radians(theta_d)) * np.cos(np.radians(phi_d))
    y = np.cos(np.radians(theta_d)) * np.sin(np.radians(phi_d))
    z = np.sin(np.radians(theta_d))
    return array([x, y, z])

### Length of Vector

In [ ]:
def vectorlength(vector):
    return np.linalg.norm(vector)

### Calculated Angular Error

In [ ]:
def calculate_angular_error(deg_e_phi, deg_e_theta):
    return np.degrees(np.arccos(np.cos(np.radians(deg_e_phi)) * np.cos(np.radians(deg_e_theta))))

### Skalarprodukt

In [ ]:
def myDot(a, b):
    dot = 0;
    it = np.nditer(a, flags=['f_index'])
    for x in it:
        dot = dot + (x * b[it.index])
        
    return dot

### Dot Angular Error

In [ ]:
def dot_angular_error_elevation(predicted_deg_vector, true_deg_vector):    
    c_predicted = spheric_cartesian_elevation(predicted_deg_vector[0], predicted_deg_vector[1])
    c_true = spheric_cartesian_elevation(true_deg_vector[0], true_deg_vector[1])
    
    len_prediction = vectorlength(c_predicted)
    len_true = vectorlength(c_true)
    
    cos_angle = np.dot(c_true, c_predicted) / len_prediction / len_true
    
    return abs(np.degrees(np.arccos(cos_angle)))

def dot_angular_error_polar(predicted_deg_vector, true_deg_vector):
    c_predicted = spheric_cartesian_polar(predicted_deg_vector[0], predicted_deg_vector[1])
    c_true = spheric_cartesian_polar(true_deg_vector[0], true_deg_vector[1])
    
    len_prediction = vectorlength(c_predicted)
    len_true = vectorlength(c_true)
    
    cos_angle = np.dot(c_true, c_predicted) / len_prediction / len_true
    
    return abs(np.degrees(np.arccos(cos_angle)))

### Automatische Optimizer Generierung aus String

In [ ]:
def make_optimizer(optimizer):
    # [Adam, Nadam, Adagrad, RMSprop]
    if optimizer == "<class 'keras.optimizers.Adam'>":
        return Adam
    elif optimizer == "<class 'tensorflow.python.keras.optimizer_v2.adam.Adam'>":
        return Adam
    elif optimizer == "<class 'keras.optimizers.Nadam'>":
        return Nadam
    elif optimizer == "<class 'keras.optimizers.Adagard'>":
        return Adagard
    elif optimizer == "<class 'keras.optimizers.RMSprop'>":
        return RMSprop
    else:
        print('ERROR::: Unspecified Optimizer')

### Convert Label_Type into suitable label names.
$\Rightarrow$ Angular / Normalized $\rightarrow$ ['Elevation', 'Azimuth']

$\Rightarrow$ Stereographic $\rightarrow$ ['S_x', 'S_y']

In [ ]:
def get_Label_Names(label_type):
    if label_type == 'Angular' or label_type == 'Normalized':
        return ['Elevation', 'Azimuth']
    elif label_type == 'Stereographic':
        return ['S_x', 'S_y']
    else:
        assert(True, 'LabelType Invalid')
        return None

### Convert String into Reduction Metric Function

In [ ]:
def get_Reduction_Metric(metric):
    
    if metric == 'custom_mae':
        return [custom_mae]
    elif metric == 'tf.keras.metrics.MeanAbsoluteError()':
        return [tf.keras.metrics.MeanAbsoluteError()]
    elif metric == 'circular_mae':
        return [circular_mae]
    elif metric == 'mean_squared_error':
        return ['mean_squared_error']
    else:
        assert(False, 'Metric yet unknown - Please modify get_Reduction_Metric to meet your requirements')
        return None

## Normierte sphärische Koordinaten

### Normierte $\rightarrow$ Sphärische

In [ ]:
def normalized_to_spheric(n_phi, n_theta):
    phi = n_phi * 180 + 180
    theta = n_theta * 45 + 45
    
    return phi, theta

### Evaluate

In [ ]:
def evaluate(eval_df, eval_predictions, save_dir = None, save_to_file = False):
    num_predictions = eval_df.shape[0]
    
    df_result = pd.DataFrame({
        'Filename': eval_df['Filename'][0:num_predictions],
        'Elevation_true': None if _STEREOGRAPHIC else eval_df['Elevation'][0:num_predictions],
        'Elevation_pred': None,
        'Elevation_err': None,
        'Azimuth_true': None if _STEREOGRAPHIC else eval_df['Azimuth'][0:num_predictions],
        'Azimuth_pred': None,
        'Azimuth_err': None,
        'Angular_MAE': None,
        'dot_angular_err_elevation': None,
        'dot_angular_err_polar': None,
        'S_x_true': eval_df['S_x'][0:num_predictions] if _STEREOGRAPHIC else None, # 'S_x' replace later
        'S_x_pred': None,
        'S_y_true': eval_df['S_y'][0:num_predictions] if _STEREOGRAPHIC else None, # 'S_y' replace later
        'S_y_pred': None,
        'S_x_err': None,
        'S_y_err': None
    })
    
    angular_calculated = ''
    stereo_calculated = ''
    
    
    if _STEREOGRAPHIC:
        angular_calculated = '*'
        stereo_calculated = ''
        
        df_result['S_x_pred'] = eval_predictions[:, 0]
        df_result['S_y_pred'] = eval_predictions[:, 1]
        
        for index, row in df_result.iterrows():
            predicted_sx = row['S_x_pred']
            predicted_sy = row['S_y_pred']
            
            true_sx = row['S_x_true']
            true_sy = row['S_y_true']
            
            # stereographische Koordinaten - Fehler
            error_sx = abs(predicted_sx - true_sx)
            error_sy = abs(predicted_sy - true_sy)
            
            df_result.at[index, 'S_x_err'] = error_sx
            df_result.at[index, 'S_y_err'] = error_sy
            
            
            # Kugelkoordinaten
            predicted_azimuth, predicted_elevation = convert_from_stereographic(predicted_sx, predicted_sy)
            true_azimuth, true_elevation = convert_from_stereographic(true_sx, true_sy)
            
            df_result.at[index, 'Elevation_true'] = true_elevation
            df_result.at[index, 'Elevation_pred'] = predicted_elevation
            df_result.at[index, 'Azimuth_true'] = true_azimuth
            df_result.at[index, 'Azimuth_pred'] = predicted_azimuth
            
            error_elevation = abs(predicted_elevation - true_elevation)
            error_azimuth = min(abs(predicted_azimuth - true_azimuth), abs(360 - abs(predicted_azimuth - true_azimuth)))
            
            df_result.at[index, 'Elevation_err'] = error_elevation
            df_result.at[index, 'Azimuth_err'] = error_azimuth
            
            
            # dot_angular_error
            predicted_direction = array([predicted_azimuth, predicted_elevation])
            true_direction = array([true_azimuth, true_elevation])
            
            error_direction_elevation = dot_angular_error_elevation(predicted_direction, true_direction)
            df_result.at[index, 'dot_angular_err_elevation'] = error_direction_elevation
            error_direction_polar = dot_angular_error_polar(predicted_direction, true_direction)
            df_result.at[index, 'dot_angular_err_polar'] = error_direction_polar
            
            
            # Angular Mean Absolute Error
            y_pred = K.constant(predicted_direction)
            y_true = K.constant (true_direction)
            mae = custom_mae(y_true, y_pred)
            df_result.at[index, 'Angular_MAE'] = mae
        
    else:
        angular_calculated = ''
        stereo_calculated = '*'
        
        df_result['Elevation_pred'] = eval_predictions[:, 0]
        df_result['Azimuth_pred'] = eval_predictions[:, 1]
        
        for index, row in df_result.iterrows():
            #print('=======================================================================')
            if(current_params['label_type'] == 'Angular'):
                predicted_azimuth = row['Azimuth_pred']
                predicted_elevation = row['Elevation_pred']
                
                true_azimuth = row['Azimuth_true']
                true_elevation = row['Elevation_true']
                
            elif(current_params['label_type'] == 'Normalized'):
                predicted_azimuth, predicted_elevation = normalized_to_spheric(row['Azimuth_pred'], row['Elevation_pred'])

                true_azimuth, true_elevation = normalized_to_spheric(row['Azimuth_true'], row['Elevation_true'])
                
            
            # azimuth and elevation error
            error_elevation = abs((predicted_elevation - true_elevation))
            df_result.at[index, 'Elevation_err'] = error_elevation

            error_azimuth = min(abs(predicted_azimuth - true_azimuth), abs(360 - abs(predicted_azimuth - true_azimuth)))
            df_result.at[index, 'Azimuth_err'] = error_azimuth
            
            
            # stereographische Koordinaten - Fehler
            sx_t, sy_t = convert_from_spheric(true_azimuth, true_elevation)
            sx_p, sy_p = convert_from_spheric(predicted_azimuth, predicted_elevation)
            
            df_result.at[index, 'S_x_true'] = sx_t
            df_result.at[index, 'S_y_true'] = sy_t
            df_result.at[index, 'S_x_pred'] = sx_p
            df_result.at[index, 'S_y_pred'] = sy_p
            
            error_sx = abs(sx_p - sx_t)
            error_sy = abs(sy_p - sy_t)
            
            df_result.at[index, 'S_x_err'] = error_sx
            df_result.at[index, 'S_y_err'] = error_sy
            
            
            # dot angular error
            predicted_direction = array([predicted_azimuth, predicted_elevation])
            true_direction = array([true_azimuth, true_elevation])
            
            error_direction_elevation = dot_angular_error_elevation(predicted_direction, true_direction)
            df_result.at[index, 'dot_angular_err_elevation'] = error_direction_elevation
            error_direction_polar = dot_angular_error_polar(predicted_direction, true_direction)
            df_result.at[index, 'dot_angular_err_polar'] = error_direction_polar
            
            
            # Angular Mean Absolute Error
            y_pred = K.constant(predicted_direction)
            y_true = K.constant (true_direction)
            mae = custom_mae(y_true, y_pred)
            df_result.at[index, 'Angular_MAE'] = mae
            #print('=======================================================================')
    
    error_elevation_avg = np.mean(df_result['Elevation_err'], axis = 0)
    error_azimuth_avg = np.mean(df_result['Azimuth_err'], axis = 0)
    error_sx_avg = np.mean(df_result['S_x_err'], axis = 0)
    error_sy_avg = np.mean(df_result['S_y_err'], axis = 0)
    
    
    # Box'n Whiskers Diagram Data - Angular Error
    e_angular_median = np.quantile(df_result['dot_angular_err_elevation'], 0.5)
    e_angular_lower_q = np.quantile(df_result['dot_angular_err_elevation'], 0.25)
    e_angular_upper_q = np.quantile(df_result['dot_angular_err_elevation'], 0.75)
    e_angular_min = np.amin(df_result['dot_angular_err_elevation'])
    e_angular_max = np.amax(df_result['dot_angular_err_elevation'])
    
    max_angular_error = df_result.loc[df_result['dot_angular_err_elevation'] == e_angular_max]
    max_angular_error_file = max_angular_error.iloc[0]['Filename']
    max_angular_error_theta_true = max_angular_error.iloc[0]['Elevation_true']
    max_angular_error_phi_true = max_angular_error.iloc[0]['Azimuth_true']
    max_angular_error_theta_pred = max_angular_error.iloc[0]['Elevation_pred']
    max_angular_error_phi_pred = max_angular_error.iloc[0]['Azimuth_pred']
    
    
    # Box'n Whiskers Diagram Data - Mean Absolute Error
    e_mae_median = np.quantile(df_result['Angular_MAE'], 0.5)
    e_mae_lower_q = np.quantile(df_result['Angular_MAE'], 0.25)
    e_mae_upper_q = np.quantile(df_result['Angular_MAE'], 0.75)
    e_mae_min = np.amin(df_result['Angular_MAE'])
    e_mae_max = np.amax(df_result['Angular_MAE'])
    
    max_mae_error = df_result.loc[df_result['Angular_MAE'] == e_mae_max]
    max_mae_error_file = max_mae_error.iloc[0]['Filename']
    max_mae_error_theta_true = max_mae_error.iloc[0]['Elevation_true']
    max_mae_error_phi_true = max_mae_error.iloc[0]['Azimuth_true']
    max_mae_error_theta_pred = max_mae_error.iloc[0]['Elevation_pred']
    max_mae_error_phi_pred = max_mae_error.iloc[0]['Azimuth_pred']
    
    # Avg Angular Error
    error_dot_angular_elevation_avg = np.mean(df_result['dot_angular_err_elevation'], axis = 0)
    angular_variance = np.var(df_result['dot_angular_err_elevation'], axis = 0)
    avg_angular_lower_mean = np.mean(df_result['dot_angular_err_elevation'][df_result['dot_angular_err_elevation'] < error_dot_angular_elevation_avg])
    avg_angular_upper_mean = np.mean(df_result['dot_angular_err_elevation'][df_result['dot_angular_err_elevation'] > error_dot_angular_elevation_avg])
    
    # Avg MAE Error
    avg_mae_mean = np.mean(df_result['Angular_MAE'])
    avg_mae_lower_mean = np.mean(df_result['Angular_MAE'][df_result['Angular_MAE'] < avg_mae_mean])
    avg_mae_upper_mean = np.mean(df_result['Angular_MAE'][df_result['Angular_MAE'] > avg_mae_mean])
    

    print('Durchschnnittlicher Fehler Elevation{}: {:.1f}'.format(angular_calculated, error_elevation_avg))
    print('Durchschnnittlicher Fehler Azimut{}: {:.1f}'.format(angular_calculated, error_azimuth_avg))
    print('Durchschnnittlicher Fehler S_x{}: {:.2f}'.format(stereo_calculated, error_sx_avg))
    print('Durchschnnittlicher Fehler S_y{}: {:.2f}'.format(stereo_calculated, error_sy_avg))
    print()
    print('Durchschnnittlicher Winkelfehler{} (Elevation): {:.1f}'.format(angular_calculated, error_dot_angular_elevation_avg))
    print('Varianz Winkelfehler{}: {:.1f}'.format(angular_calculated, angular_variance))
    print()
    print('Box-Whisker Angular: max - {:.1f}, upper - {:.1f}, median - {:.1f}, lower - {:.1f}, min - {:.1f}'.format(e_angular_max, e_angular_upper_q, e_angular_median, e_angular_lower_q, e_angular_min))
    print('Maxium Angular Error on: {}, True ({:.1f}, {:.1f}), Pred ({:.1f}, {:.1f})'.format(max_angular_error_file, max_angular_error_phi_true, max_angular_error_theta_true, max_angular_error_phi_pred, max_angular_error_theta_pred))
    print('Box-Whisker MAE: max - {:.1f}, upper - {:.1f}, median - {:.1f}, lower - {:.1f}, min - {:.1f}'.format(e_mae_max, e_mae_upper_q, e_mae_median, e_mae_lower_q, e_mae_min))
    print('Maxium MAE Error on: {}, True ({:.1f}, {:.1f}), Pred ({:.1f}, {:.1f})'.format(max_mae_error_file, max_mae_error_phi_true, max_mae_error_theta_true, max_mae_error_phi_pred, max_mae_error_theta_pred))
    print()
    print('Avg Angular Range: lower - {:.1f}, avg - {:.1f}, upper - {:.1f}'.format(avg_angular_lower_mean, error_dot_angular_elevation_avg, avg_angular_upper_mean))
    print('Avg MAE Range: lower - {:.1f}, avg - {:.1f}, upper - {:.1f}'.format(avg_mae_lower_mean, avg_mae_mean, avg_mae_upper_mean))
    
        
    df_avg = pd.DataFrame({
        'Avg_Elevation_Err': [error_elevation_avg],
        'Avg_Azimuth_Err': [error_azimuth_avg],
        'Avg_Angular_Err': [error_dot_angular_elevation_avg],
        'Angular_Variance': [angular_variance],
        'Avg_Sx_Err': [error_sx_avg],
        'Avg_Sy_Err': [error_sy_avg],
        'Avg_Inference_Time': [image_time],
        'box_angular_medium': [e_angular_median],
        'box_angular_lower': [e_angular_lower_q],
        'box_angular_upper': [e_angular_upper_q],
        'box_angular_min': [e_angular_min],
        'box_angular_max': [e_angular_max],
        'box_mae_medium': [e_mae_median],
        'box_mae_lower': [e_mae_lower_q],
        'box_mae_upper': [e_mae_upper_q],
        'box_mae_min': [e_mae_min],
        'box_mae_max': [e_mae_max],
        'avg_angular_lower_mean': [avg_angular_lower_mean],
        'avg_angular_upper_mean': [avg_angular_upper_mean],
        'avg_mae_mean': [avg_mae_mean],
        'avg_mae_lower_mean': [avg_mae_lower_mean],
        'avg_mae_lower_mean': [avg_mae_upper_mean],
        'max_angular_error_file': [max_angular_error_file],
        'max_angular_error_theta_true': [max_angular_error_theta_true],
        'max_angular_error_phi_true': [max_angular_error_phi_true],
        'max_angular_error_theta_pred': [max_angular_error_theta_pred],
        'max_angular_error_phi_pred': [max_angular_error_phi_pred],
        'max_mae_error_file': [max_mae_error_file],
        'max_mae_error_theta_true': [max_mae_error_theta_true],
        'max_mae_error_phi_true': [max_mae_error_phi_true],
        'max_mae_error_theta_pred': [max_mae_error_theta_pred],
        'max_mae_error_phi_pred': [max_mae_error_phi_pred]
    })

    if save_to_file:
        df_result.to_csv(save_dir + 'Prognosen_ErrE_{}_ErrA_{}.csv'.format('%.2f'%error_elevation_avg, '%.2f'%error_azimuth_avg), index=False)
    
    model_path, current_model = ntpath.split(model)
    evaluation_path = model_path + '\\Evaluation\\'
    if(not os.path.exists(evaluation_path)):
        os.makedirs(evaluation_path)
    df_result.to_csv(evaluation_path + 'Model-{}_Testset-{}_Prediction_Results.csv'.format(net_index, trainingset_to_string(test_set)), index=False)
    df_avg.to_csv(evaluation_path + 'Model-{}_Testset-{}_Average_Results.csv'.format(net_index, trainingset_to_string(test_set)), index=False)
    
    return df_result, df_avg

# Test the Model <a name = "setup"></a>

## Hilfsdaten

In [ ]:
mse_results = []

## Evaluation Setup
<p><a href = #Top>Top</a> 
<p><a href = #store>Save File</a>

#### Required format of parameters parameter for _model_predict_ (...)

In [ ]:
p = {
    'dataset_to_use':[],
    'model_to_load':[],
    'dataset_name':'combined_dataset',
    # --------------------------------
    'optimizer':[],
    'learning_rate':[],
    'first_neuron':[],
    'dropout_rate':[],
    'activation_function':[],
    'leaky_ReLU_alpha':[],
    'hidden_layers':[],
    # --------------------------------
    'label_type':['Angular'],
    'loss_function':[],
    'reduction_metric':[],
    'monitor_value':[]
}

In [ ]:
run = 'SYNTH'
loss = 'MSE'
dataset_name = '2020-05-28' # -no_shadow
net_index = 31
synth_samples = 80000
real_samples = 640
mixed_samples = 80000
talos_results = 'Talos_Results_Fine_Idx{}.csv'.format(net_index)

#APPENDIX = 'Stereographic'

_note = '_Custom-MAE' # _Custom-MAE, _cmae-base, _Verification

#FUNCTION_OVERRIDE = ['mean_squared_error', [custom_mae], 'val_custom_mae'] # e. g. ['mean_squared_error', [circular_mae], 'val_circular_mae']


if p['label_type'][0] == 'Stereographic':
    _CSV_SYNTH_FILE_NAME = 'images_synthetisch_stereographic.csv'
    _CSV_REAL_FILE_NAME = 'images_real_stereographic.csv'
    _STEREOGRAPHIC = True
    #label_type = LabelType.STEREOGRAPHIC
elif p['label_type'][0] == 'Angular':
    _CSV_SYNTH_FILE_NAME = 'images_synthetisch.csv'
    _CSV_REAL_FILE_NAME = 'images_real.csv'
    _STEREOGRAPHIC = False
    #label_type = LabelType.ANGULAR
elif p['label_type'][0] == 'Normalized':
    _CSV_SYNTH_FILE_NAME = 'images_synthetisch_normalized.csv'
    _CSV_REAL_FILE_NAME = 'images_real_normalized.csv'
    _STEREOGRAPHIC = False
    #label_type = LabelType.ANGULAR
else:
    assert(True, 'Appendix Invalid')


network_path = '..\\output\\{}_Regression_{}\\{}_{}_Top_1{}\\{}\\'
    
syn_trained_model_dir = network_path.format(run, loss, dataset_name, p['label_type'][0], _note, 'Synth_TD')
real_trained_model_dir = network_path.format(run, loss, dataset_name, p['label_type'][0], _note, 'Real_TD')
mixed_trained_model_dir = network_path.format(run, loss, dataset_name, p['label_type'][0], _note, 'Mixed_TD')

p['dataset_name'] = dataset_name
_IMAGE_DIR = '..\\dataset\\{}\\'.format(p['dataset_name'])

_CSV_FILE = _IMAGE_DIR + _CSV_SYNTH_FILE_NAME
_CSV_FILE_REAL = _IMAGE_DIR + _CSV_REAL_FILE_NAME

In [ ]:
test_set = TrainingSet.SYNTHETIC

#####  Trainiert mit: Synthetische Daten

In [ ]:
network_file = syn_trained_model_dir + talos_results
dataframe = pd.read_csv(network_file)

dataframe.head()

In [ ]:
label_type = None
loss_function = None
reduction_metric = None
monitor_value = None

In [ ]:
model = syn_trained_model_dir + 'CNN_Base_{}_Model_and_Weights_{}.hdf5'.format(net_index, synth_samples)
current_params = load_params(dataframe, test_set, model, dataset_name, label_type, loss_function, reduction_metric, monitor_value)

In [ ]:
predictions, df_test, duration, image_time = model_predict(current_params)

In [ ]:
result_df, avg_df = evaluate(df_test, predictions)
mse_results.append([network_file, result_df, current_params['dataset_to_use'], avg_df, label_type])

##### Trainiert mit: Reale Daten

In [ ]:
network_file = real_trained_model_dir + talos_results
dataframe = pd.read_csv(network_file)

In [ ]:
model = real_trained_model_dir + 'CNN_Base_{}_Model_and_Weights_{}.hdf5'.format(net_index, real_samples)
current_params = load_params(dataframe, test_set, model, dataset_name, label_type, loss_function, reduction_metric, monitor_value)

In [ ]:
predictions, df_test, duration, image_time = model_predict(current_params)

In [ ]:
result_df, avg_df = evaluate(df_test, predictions)
mse_results.append([network_file, result_df, current_params['dataset_to_use'], avg_df, label_type])

##### Trainiert mit: Gemischte Daten

In [ ]:
network_file = mixed_trained_model_dir + talos_results
dataframe = pd.read_csv(network_file)

In [ ]:
model = mixed_trained_model_dir + 'CNN_Base_{}_Model_and_Weights_{}.hdf5'.format(net_index, mixed_samples)
current_params = load_params(dataframe, test_set, model, dataset_name, label_type, loss_function, reduction_metric, monitor_value)

In [ ]:
predictions, df_test, duration, image_time = model_predict(current_params)

In [ ]:
result_df, avg_df = evaluate(df_test, predictions)
mse_results.append([network_file, result_df, current_params['dataset_to_use'], avg_df, label_type])

## Reale Testdaten

In [ ]:
test_set = TrainingSet.REAL

##### Trainiert mit: Synthetische Daten

In [ ]:
network_file = syn_trained_model_dir + talos_results
dataframe = pd.read_csv(network_file)

In [ ]:
model = syn_trained_model_dir + 'CNN_Base_{}_Model_and_Weights_{}.hdf5'.format(net_index, synth_samples)
current_params = load_params(dataframe, test_set, model, dataset_name, label_type, loss_function, reduction_metric, monitor_value)

In [ ]:
predictions, df_test, duration, image_time = model_predict(current_params)

In [ ]:
result_df, avg_df = evaluate(df_test, predictions)
mse_results.append([network_file, result_df, current_params['dataset_to_use'], avg_df, label_type])

##### Trainiert mit: Reale Daten

In [ ]:
network_file = real_trained_model_dir + talos_results
dataframe = pd.read_csv(network_file)

In [ ]:
model = real_trained_model_dir + 'CNN_Base_{}_Model_and_Weights_{}.hdf5'.format(net_index, real_samples)
current_params = load_params(dataframe, test_set, model, dataset_name, label_type, loss_function, reduction_metric, monitor_value)

In [ ]:
predictions, df_test, duration, image_time = model_predict(current_params)

In [ ]:
result_df, avg_df = evaluate(df_test, predictions)
mse_results.append([network_file, result_df, current_params['dataset_to_use'], avg_df, label_type])

##### Traininert mit: Gemischte Daten

In [ ]:
network_file = mixed_trained_model_dir + talos_results
dataframe = pd.read_csv(network_file)

In [ ]:
model = mixed_trained_model_dir + 'CNN_Base_{}_Model_and_Weights_{}.hdf5'.format(net_index, mixed_samples)
current_params = load_params(dataframe, test_set, model, dataset_name, label_type, loss_function, reduction_metric, monitor_value)

In [ ]:
predictions, df_test, duration, image_time = model_predict(current_params)

In [ ]:
result_df, avg_df = evaluate(df_test, predictions)
mse_results.append([network_file, result_df, current_params['dataset_to_use'], avg_df, label_type])

# Evaluation <a name = "store">
<p></a><a href = #Top>Top</a>
<p><a href = #setup>Setup</a>

In [ ]:
eval_dir = '..\\output\\{}_Regression_{}\\Graphical_Evaluation\\'.format(run, loss)

if(not os.path.exists(eval_dir)):
    os.makedirs(eval_dir)
else:
    input('Directory >>| {} |<< existiert bereits. Fortsetzen auf eigene Gefahr! (Weiter mit Enter)'.format(eval_dir))

In [ ]:
import pickle

with open(eval_dir + '{}_Net{}_{}{}_Results.pickle'.format(dataset_name, net_index, p['label_type'][0], _note), "wb") as fp:   #Pickling
    pickle.dump(mse_results, fp) # uncomment if you are REALLY sure to overwrite the file

with open(eval_dir + '{}_Net{}_{}{}_Results.pickle'.format(dataset_name, net_index, p['label_type'][0], _note), "rb") as fp:   # Unpickling
    b = pickle.load(fp)